In [1]:
import numpy as np
import pandas as pd
from ase.io.xyz import read_xyz
from ase.io import read
from io import StringIO
import os
import schnetpack as spk
from torch.optim import Adam
import matplotlib.pyplot as plt
import schnetpack.train as trn
from schnetpack import AtomsData
import torch
from sklearn.model_selection import KFold

In [2]:
freesolv_file = 'data/FreeSolv_with_3D.csv'
freesolv_data = pd.read_csv(freesolv_file)

In [3]:
freesolvmod = "./FreeSolvModel"
if not os.path.exists('FreeSolvModel'):
    os.makedirs(freesolvmod)

In [4]:
atoms = freesolv_data['xyz'].map(lambda x: next(read_xyz(StringIO(x), slice(None))))
atoms = [a for a in atoms]

In [5]:
freesolv_expt = np.array(freesolv_data["expt"],dtype=float)

In [6]:
property_list = []
for f in freesolv_expt:
    
    property_list.append(
        {'expt': float(f)}
    )

print('Properties:', property_list)

Properties: [{'expt': -11.01}, {'expt': -4.87}, {'expt': 1.83}, {'expt': -5.45}, {'expt': -4.21}, {'expt': -6.27}, {'expt': 2.34}, {'expt': -3.92}, {'expt': 1.58}, {'expt': -4.62}, {'expt': -1.96}, {'expt': -3.88}, {'expt': -4.33}, {'expt': -2.49}, {'expt': -2.55}, {'expt': -4.78}, {'expt': -1.46}, {'expt': -2.22}, {'expt': -5.85}, {'expt': 2.88}, {'expt': -6.34}, {'expt': -6.1}, {'expt': -5.48}, {'expt': -4.1}, {'expt': -4.1}, {'expt': -3.52}, {'expt': -4.47}, {'expt': -2.74}, {'expt': -3.43}, {'expt': -7.81}, {'expt': -0.8}, {'expt': 0.93}, {'expt': -6.78}, {'expt': -9.29}, {'expt': -4.07}, {'expt': -5.91}, {'expt': -2.13}, {'expt': -3.64}, {'expt': -2.68}, {'expt': 3.13}, {'expt': -10.0}, {'expt': 2.06}, {'expt': -8.11}, {'expt': -1.12}, {'expt': -23.62}, {'expt': -2.83}, {'expt': -4.68}, {'expt': -5.56}, {'expt': -3.65}, {'expt': -5.49}, {'expt': 3.12}, {'expt': -4.42}, {'expt': -6.2}, {'expt': -6.96}, {'expt': -0.16}, {'expt': -0.25}, {'expt': -9.3}, {'expt': -4.63}, {'expt': -10.

In [7]:
new_dataset = AtomsData(os.path.join(freesolvmod, 'FreeSolv_SchNet_dataset.db'), available_properties=['expt'])
new_dataset.add_systems(atoms, property_list)

In [8]:
print('Number of reference calculations:', len(new_dataset))
print('Available properties:')

for p in new_dataset.available_properties:
    print('-', p)
print()

example = new_dataset[0]
print('Properties of molecule with id 0:')

for k, v in example.items():
    print('-', k, ':', v.shape)

Number of reference calculations: 10914
Available properties:
- expt

Properties of molecule with id 0:
- expt : torch.Size([1])
- _atomic_numbers : torch.Size([26])
- _positions : torch.Size([26, 3])
- _cell : torch.Size([3, 3])
- _neighbors : torch.Size([26, 25])
- _cell_offset : torch.Size([26, 25, 3])
- _idx : torch.Size([1])


In [9]:
train, val, test = spk.train_test_split(
        data=new_dataset,
        num_train=500,
        num_val=100,
        split_file=None#os.path.join(freesolvmod, "freesolv_split.npz"),
    )

In [10]:
train_loader = spk.AtomsLoader(train, batch_size=100, shuffle=True)
val_loader = spk.AtomsLoader(val, batch_size=100)

In [11]:
schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=5,
    cutoff=4., cutoff_network=spk.nn.cutoff.CosineCutoff
)

In [12]:
output = spk.atomistic.Atomwise(n_in=30, property='expt')

model = spk.AtomisticModel(representation=schnet, output_modules=output)

In [13]:
optimizer = Adam(model.parameters(), lr=1e-2)

In [14]:
loss = trn.build_mse_loss(['expt'])

metrics = [spk.metrics.MeanAbsoluteError('expt')]

hooks = [
    trn.CSVHook(log_path=freesolvmod, metrics=metrics),
    trn.ReduceLROnPlateauHook(
        optimizer,
        patience=5, factor=0.8, min_lr=1e-6,
        stop_after_min=True
    )
]

trainer = trn.Trainer(
    model_path=freesolvmod,
    model=model,
    hooks=hooks,
    loss_fn=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
)

In [15]:
device = "cpu" # change to 'cpu' if gpu is not available, change to cuda if gpu is
n_epochs = 25 # takes about 10 min on a notebook GPU. reduces for playing around

print('training')
trainer.train(device=device, n_epochs=n_epochs)

training


AttributeError: 'float' object has no attribute 'copy'

In [18]:
best_model = torch.load(os.path.join(freesolvmod, 'best_model'))

#train, val, test = spk.train_test_split(
#        data=new_dataset,
#        split_file=os.path.join(freesolvmod, "split.npz"),
#    )

test_loader = spk.AtomsLoader(test, batch_size=100)

energy_error = 0.0
forces_error = 0.0

for count, batch in enumerate(test_loader):
    # move batch to GPU, if necessary
    batch = {k: v.to(device) for k, v in batch.items()}

    # apply model
    pred = best_model(batch)

    # calculate absolute error of energies
    tmp_energy = torch.sum(torch.abs(pred['expt'] - batch['expt']))
    tmp_energy = tmp_energy.detach().cpu().numpy() # detach from graph & convert to numpy
    energy_error += tmp_energy

    # calculate absolute error of forces, where we compute the mean over the n_atoms x 3 dimensions
    tmp_forces = torch.sum(
        torch.mean(torch.abs(pred[MD17.forces] - batch[MD17.forces]), dim=(1,2))
    )
    tmp_forces = tmp_forces.detach().cpu().numpy() # detach from graph & convert to numpy
    forces_error += tmp_forces

    # log progress
    percent = '{:3.2f}'.format(count/len(test_loader)*100)
    print('Progress:', percent+'%'+' '*(5-len(percent)), end="\r")

energy_error /= len(test)
forces_error /= len(test)

print('\nTest MAE:')
print('    energy: {:10.3f} kcal/mol'.format(energy_error))
print('    forces: {:10.3f} kcal/mol/\u212B'.format(forces_error))

AttributeError: 'float' object has no attribute 'copy'